<a href="https://colab.research.google.com/github/vishnubob/AoC2021/blob/main/AoC_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import re
import requests
import numpy as np

cache_dir = "/content/drive/MyDrive/AoC2021/"

session_fn = f"{cache_dir}/session_id"
with open(session_fn) as fh:
    session_id = fh.read().strip()

def aoc_input(day, split=True):
    url = f"https://adventofcode.com/2021/day/{day}/input"
    cache_fn = f"{cache_dir}/d{day}.dat"
    if not os.path.exists(cache_fn):
        cookies = {'session': session_id}
        resp = requests.get(url, cookies=cookies)
        data = resp.content.decode()
        with open(cache_fn, 'w') as fh:
            fh.write(data)
    with open(cache_fn) as fh:
        data = fh.read()
    data = data.strip()
    if split:
        data = data.split('\n')
    return data

In [ ]:
inp = aoc_input(1)

# day 1, puzzle 1
inp = np.array([int(x) for x in inp])
cnt = np.sum((inp[1:] - inp[:-1]) > 0)
print(f"Day 1, puzzle 1: {cnt}")

# day 1, puzzle 2
strides = (inp.strides[0], inp.strides[0])
shape = (len(inp) - 2, 3)
inp2 = np.lib.stride_tricks.as_strided(inp, shape=shape, strides=strides)
inp2 = np.sum(inp2, axis=1)
cnt = np.sum((inp2[1:] - inp2[:-1]) > 0)
print(f"Day 1, puzzle 2: {cnt}")

Day 1, puzzle 1: 1316
Day 1, puzzle 2: 1344


In [ ]:
inp = aoc_input(2)

# day 2, puzzle 1
axis_map = ["forward", "down", "up"]
axis_id = np.eye(3, dtype=np.int)
inp = [line.split(' ') for line in inp]
cols = np.array([int(val) * axis_id[axis_map.index(dr)] for (dr, val) in inp])
sums = np.sum(cols, axis=0)
prod = sums[0] * (sums[1] - sums[2])
print(f"Day 2, puzzle 1: {prod}")

# day 2, puzzle 2
aim = np.cumsum((cols[:, 1] + -cols[:, 2]))
prod = np.sum(cols[:, 0]) * np.sum(cols[:, 0] * aim)
print(f"Day 2, puzzle 2: {prod}")


Day 2, puzzle 1: 1499229
Day 2, puzzle 2: 1340836560


In [ ]:
inp = aoc_input(3)

# day 3, puzzle 1
n_items = len(inp)
shape = (n_items, -1)
inp = str.join('', inp).encode()
inp = (np.frombuffer(inp, dtype=np.uint8).reshape(shape) - ord('0')).astype(np.int)
bits = (2 ** np.arange(inp.shape[1]))[::-1]
gamma = np.sum(bits * (np.sum(inp.T, axis=1) > (inp.shape[0] // 2)))
prod = gamma * (~gamma & (2 ** inp.shape[1] - 1))
print(f"Day 3, puzzle 1: {prod}")

# day 3, puzzle 2
t_col = lambda tbl, col: (np.sum(tbl.T, axis=1) >= ((tbl.shape[0] + 1) // 2))[col]
t_mask = lambda tbl, col: tbl[tbl.T[col, :] == t_col(tbl, col)]
t_find = lambda tbl, col: tbl if tbl.shape[0] == 1 else t_find(t_mask(tbl, col), col + 1)
o2 = np.sum(bits * t_find(inp, 0))
t_col = lambda tbl, col: (np.sum(tbl.T, axis=1) <= ((tbl.shape[0] - 1) // 2))[col]
co2 = np.sum(bits * t_find(inp, 0))
prod = o2 * co2
print(f"Day 3, puzzle 2: {prod}")

Day 3, puzzle 1: 2648450
Day 3, puzzle 2: 2845944


In [165]:
inp = aoc_input(4, split=False)

# day 4, puzzle 1

n_items = len(inp)
inp = inp.strip().split('\n\n')
n_boards = len(inp) - 1
calls = np.fromiter(map(int, inp[0].split(',')), dtype=np.int)
inp = map(int, re.sub('\s+', ' ', str.join(' ', inp[1:]).replace('\n', ' ')).split(' '))
inp = np.fromiter(inp, dtype=np.int)
dim = int((len(inp) / n_boards) ** 0.5)
inp = inp.reshape((n_boards, dim, dim))

def mark_and_check(call, inp, marks):
    bingo = -inp.shape[-1]
    marks[inp == call] = -1
    check = \
        (np.sum(marks.T, axis=0) == bingo) \
      | (np.sum(marks.T, axis=1) == bingo)
    if np.any(check):
        idx = np.argmax(check)
        winner = np.unravel_index(idx, check.shape)[1]
        idx = marks[winner] == 0
        return (winner, np.sum(inp[winner][idx]) * call)

def run_game(calls, inp, marks):
    res = mark_and_check(calls[0], inp, marks)
    if res is None:
        return run_game(calls[1:], inp, marks)
    return res[-1]

marks = np.zeros_like(inp)
prod = run_game(calls, inp, marks)
print(f"Day 4, puzzle 1: {prod}")

# day 4, puzzle 2

def find_last(calls, inp, marks, last_prod=None):
    if len(calls) == 0:
        return last_prod
    res = mark_and_check(calls[0], inp, marks)
    if res is not None:
        (winner, last_prod) = res
        inp = np.delete(inp, winner, 0)
        marks = np.delete(marks, winner, 0)
        return find_last(calls, inp, marks, last_prod)
    else:
        return find_last(calls[1:], inp, marks, last_prod)

marks = np.zeros_like(inp)
prod = find_last(calls, inp, marks)
print(f"Day 4, puzzle 2: {prod}")

Day 4, puzzle 1: 49860
Day 4, puzzle 2: 24628
